In [1]:
from proj1_helpers import *
from implementations import *
from auxiliary import *
import pickle
import matplotlib.pyplot as plt
import datetime
import random

%matplotlib inline
%reload_ext autoreload
%autoreload 2

## Data loading


### Training sets

In [2]:
easy_data_set = False # changer pour sélectionner un set de dim (1000,2) ou le vrai set

if easy_data_set:
    ids, y, X = load_easy_data(sub_sample=False)
else:
    ids, y, X = load_csv_data('../data/train.csv')
    # Only 100% clean data
    ranges = [(1,4), (7,12), (13,23), (29,30)]
    keep_idx = build_idx(ranges)
    X = X[:,keep_idx]

x, mean_x, std_x = standardize(X)
tx = np.c_[np.ones((y.shape[0], 1)), x]
tx = build_poly(x, 1)

N = x.shape[0]
D = x.shape[1]


In [8]:
# to chekc that we eliminated all unecessary features
clean_data = [x for x in X if not -999. in x]
len(clean_data)/len(X)*100 # percent

100.0

array([ 1.,  0.,  0., ...,  1.,  0.,  0.])

### Testing sets

In [4]:
# takes time, run only when needed
ids_ukn, y_ukn, X_ukn = load_csv_data('../data/test.csv')

ranges = [(1,4), (7,12), (13,23), (29,30)]
keep_idx = build_idx(ranges)
X_ukn = X_ukn[:,keep_idx]

x_ukn, mean_x_ukn, std_x_ukn = standardize(X_ukn)
tx_ukn = build_poly(x_ukn, 1)

# Training
Différents types de training possibles. Normalement, seul le logistic regression a du sens. Les autres sont là pour pouvoir tester les diverses fonctions à fournir dans `implementation.py`.

In [5]:
losses_per_algorithm = {}

## Least squares

In [7]:
w_LS, loss = least_squares(y,tx)
print(loss)
losses_per_algorithm['LS'] = loss

0.0881281775747


## Least squares (GD)

In [ ]:
# w_initial = w_LS
w_initial = np.array([0]*(D+1))
max_iters = 100
gammas = [1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0.2, 0.5]
losses = []
ws = [] 

for gamma in gammas:
    w, loss = least_squares_GD(y, tx, w_initial, max_iters, gamma)
    losses.append(loss)
    ws.append(w)

In [ ]:
print(losses)
w_LS_GD = ws[8]

## Least squares (SGD)

In [10]:
w_initial = np.array([0]*(D+1))
max_iters = 1000
gammas = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0.2, 0.4, 0.5]
batch_size = 10
losses = []
ws = []

for gamma in gammas:
    sgd_ws, sgd_losses = least_squares_SGD(y, tx, w_initial, batch_size, max_iters, gamma)
    losses.append(sgd_losses)
    ws.append(sgd_ws)

KeyboardInterrupt: 

In [ ]:
print(losses)
print(len(ws))
w_LS_SGD = ws[7]

## Polynomial regression

In [ ]:
"""Constructing the polynomial basis function expansion of the data,
   and then running least squares regression."""
# define parameters
degrees = [1, 3, 7, 12]
losses = []
ws = []
    
phi = build_poly(x, 2)  # form the data to do polynomial regression.
print(x[0,:])
print(phi[0,:])
    
for ind, degree in enumerate(degrees):
    phi = build_poly(x, degree)  # form the data to do polynomial regression.
    # least square and calculate RMSE
        
    w_degree_LS, loss_degree_LS = least_squares(y, phi)
        
    losses.append(loss_degree_LS)
    ws.append(w_degree_LS)
    print("Processing {i}th experiment, degree={d}, loss={loss}".format(
           i=ind + 1, d=degree, loss=loss_degree_LS))


In [ ]:
w_degree_LS = ws[np.argmin(losses)]
print(np.argmin(losses))


## Ridge Regression
A tester!

In [ ]:
lambdas = np.logspace(-5, 0, 15)

ratio = 0.5 
degree = 7

# split the data, and return train and test data
x_train, x_test, y_train, y_test = split_data(x, y, ratio)
    
# form train and test data with polynomial basis function
phi_train = build_poly(x_train, degree)
phi_test = build_poly(x_test, degree)
    
rmse_tr = []
rmse_te = []
for ind, lambda_ in enumerate(lambdas):
    # ridge regression with a given lambda
    w, loss = ridge_regression(y_train, phi_train, lambda_)
    rmse_tr.append(compute_loss(y_train,phi_train,w))
    rmse_te.append(compute_loss(y_test,phi_test,w))

    print("proportion={p}, degree={d}, lambda={l:.3f}, Training RMSE={tr:.3f}, Testing RMSE={te:.3f}".format(
            p=ratio, d=degree, l=lambda_ ,tr=rmse_tr[ind], te=rmse_te[ind]))

## Logistic regression

In [ ]:
w_initial = np.array([0]*(D+1))
max_iters = 1000
gammas = [1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0.2, 0.5]
gammas = [0.01]
losses = []
ws = []

for gamma in gammas:
    w, loss = logistic_regression(y, tx, w_initial, max_iters, gamma)
    losses.append(loss)
    ws.append(w)


In [ ]:
print(losses)
w_LR = ws[0]
print(w_LR)

# Evaluate for test set
Sert à afficher le nombre d'erreurs

In [ ]:
tx = build_poly(x, 1)
nb_errors, error_mean = compute_classification_error(y, tx, w_reg_log_reg)
print(nb_errors)
print(error_mean)

In [ ]:
print(w_LS)
print(w_LS_GD)
print(w_LS_SGD)

## Cross validation with least squares
Je l'ai fait pour la beauté du geste, mais normalement il ne faudra pas utiliser ce bloc de code

In [ ]:
def train_test_split_demo(x, y, degree, ratio, seed):
    """polynomial regression with different split ratios and different degrees."""
    # split the data, and return train and test data: TODO
    x_train, x_test, y_train, y_test = split_data(x, y, ratio)

    # form train and test data with polynomial basis function: TODO
    phi_train = build_poly(x_train, degree)
    phi_test = build_poly(x_test, degree)
    
    # calcualte weight through least square.: TODO
    w,loss = least_squares(y_train, phi_train)

    # calculate RMSE for train and test data,
    # and store them in rmse_tr and rmse_te respectively: TODO
    loss_tr = compute_loss(y_train,phi_train,w)
    loss_te = compute_loss(y_test,phi_test,w)
    
    print("proportion={p}, degree={d}, Training RMSE={tr:.3f}, Testing RMSE={te:.3f}".format(
          p=ratio, d=degree, tr=loss_tr, te=loss_te))


In [ ]:
seed = 6
degrees = [1, 3, 7, 12]
split_ratios = [0.9, 0.5, 0.1]

for split_ratio in split_ratios:
    for degree in degrees:
        train_test_split_demo(x, y, degree, split_ratio, seed)

## Cross validation with logistic regression
Les choses sérieuses commencent ici avec des tests sur le logistic regression. Cependant, on peut directement passer à la section suivante, qui est une généralisation (il y a le "lambda" en plus) de ce code-ci.

In [ ]:
def logistic_regression_split(x, y, degree, ratio, gamma, seed):
    """polynomial regression with different split ratios and different degrees."""
    # split the data, and return train and test data: TODO
    x_train, x_test, y_train, y_test = split_data(x, y, ratio, seed)

    # form train and test data with polynomial basis function: TODO
    phi_train = build_poly(x_train, degree)
    phi_test = build_poly(x_test, degree)
    
    w_initial = np.array([0]*(phi_train.shape[1]))
    max_iters = 1000

    
    # calcualte weight through least square.: TODO
    w,loss_tr = logistic_regression(y_train, phi_train, w_initial, max_iters, gamma)
    
    loss_te = compute_logistic_loss(y_test, phi_test, w)
    # compute error and loss for train and test data
    error, ratio_error_train = compute_classification_error(y_train, phi_train, w)
    error, ratio_error_test = compute_classification_error(y_test, phi_test, w)

    
    print("proportion={p}, degree={d}, gamma={g}, Train loss, err_ratio=({tr:.3f},{er_tr:.3f}), Test loss, err_ratio=({te:.3f},{er_te:.3f})".format(
          p=ratio, d=degree, g=gamma, tr=loss_tr, te=loss_te, er_tr=ratio_error_train, er_te=ratio_error_test))
    return w, ratio_error_train, ratio_error_test


In [ ]:
seed = 6
degrees = [1, 3, 7, 12]
split_ratios = [0.9, 0.8, 0.5, 0.1]
gammas = [1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0.2, 0.5]

ws = []
ratio_err_trains = []
ratio_err_tests = []

for split_ratio in split_ratios:
    for degree in degrees:
        for gamma in gammas:
            w, ratio_error_train, ratio_error_test = logistic_regression_split(x, y, degree, split_ratio, gamma, seed)
            ws.append(w)
            ratio_err_trains.append(ratio_error_train)
            ratio_err_tests.append(ratio_error_test)
            

## Cross validation with regularized logistic regression
Voilà le code utilisé pour la première soumission. Attention: le code est long... pour bien faire il faudrait mettre un grand nombre de paramètres et faire tourner le code pendant ~1heure (brute-force, mais il se peut que ça fonctionne bien)

In [ ]:
def reg_logistic_regression_split(x, y, degree, ratio, gamma, lambda_, seed):
    """polynomial regression with different split ratios and different degrees."""
    # split the data, and return train and test data: TODO
    x_train, x_test, y_train, y_test = split_data(x, y, ratio, seed)

    # form train and test data with polynomial basis function: TODO
    phi_train = build_poly(x_train, degree)
    phi_test = build_poly(x_test, degree)
    
    w_initial = np.array([0]*(phi_train.shape[1]))
    max_iters = 1000

    
    # calcualte weight through logistic regression
    w, loss_tr = reg_logistic_regression(y_train, phi_train, lambda_, w_initial, max_iters, gamma, SGD=False)
    
    loss_te = compute_logistic_loss(y_test, phi_test, w)
    # compute error and loss for train and test data
    error, ratio_error_train = compute_classification_error(y_train, phi_train, w)
    error, ratio_error_test = compute_classification_error(y_test, phi_test, w)

    
    print("prop={p}, deg={d}, g={g:.3f}, l={l:.3f}, Train loss, err_ratio=({tr:.3f},{er_tr:.3f}), Test loss, err_ratio=({te:.3f},{er_te:.3f})".format(
          p=ratio, d=degree, g=gamma, l=lambda_, tr=loss_tr, te=loss_te, er_tr=ratio_error_train, er_te=ratio_error_test))
    return w, ratio_error_train, ratio_error_test


In [ ]:
seed = 6
degrees = [1, 7]
split_ratios = [0.9]
gammas = [1e-9, 1e-5, 1e-1, 0.5]
lambdas = np.logspace(-5, 0, 2)

ws = []
ratio_err_trains = []
ratio_err_tests = []

for split_ratio in split_ratios:
    for degree in degrees:
        for gamma in gammas:
            for lambda_ in lambdas:
                w, ratio_error_train, ratio_error_test = reg_logistic_regression_split(x, y, degree, split_ratio, gamma, lambda_, seed)
                ws.append(w)
                ratio_err_trains.append(ratio_error_train)
                ratio_err_tests.append(ratio_error_test)
            

In [ ]:
w_reg_log_reg = ws[np.argmin(ratio_err_tests)]
np.argmin(ratio_err_tests)

# Perform estimation on evaluation set
Une fois qu'on a le bon vecteur de poids, il suffit de runner ce code pour avoir le fichier de sortie.

In [ ]:
y_ukn = predict_labels(w_reg_log_reg, tx_ukn)
y_ukn[y_ukn==0] = -1
create_csv_submission(ids_ukn, y_ukn, 'submission.csv')